In [1]:
!pip install qiskit
!pip install pylatexenc
!pip install matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136817 sha256=ed3359974a99771168c1a059cd391900f9522d2d7631a6969598120a910eb510
  Stored in directory: /root/.cache/pip/wheels/d3/31/8b/e09b0386afd80cfc556c00408c9aeea5c35c4d484a9c762fd5
Successfully built pylatexenc


In [16]:
from qiskit import QuantumCircuit, transpile
from qiskit.circuit import ParameterVector, Parameter
from qiskit.providers.fake_provider import GenericBackendV2
import random as random
import numpy as np
import matplotlib.pyplot as plt


qubits = 2
layers = 2
shots = 1024
trials = 100
epsilon = np.pi/4
learning_rate = .1

# ------------------------------------------------------------------------------
# The following hamiltonian defined by:
measurements = ['ZZ', 'XX', 'YY']
coeffs = [1.0, 0.5, 0.5]
# has lowest energy -2.
# Let's find it.

# The algorithm works as follows.
# ------------------------------------------------------------------------------


# Build the base circuit without measurements
def BuildBaseCircuit(params):
  cirq = QuantumCircuit(qubits)
  param_idx = 0
  for layer in range(layers):
    # Apply rotations
    for qubit in range(qubits):
      cirq.rx(params[param_idx], qubit)
      param_idx += 1
      cirq.ry(params[param_idx], qubit)
      param_idx += 1
    # Apply entanglement
    for qubit in range(qubits - 1):
      cirq.cx(qubit, qubit+1)
  display(cirq.draw())
  return cirq

def ApplyPauliMeasurements(base_cirq, paramValues, paulis):
  cirq = base_cirq.assign_parameters(paramValues)
  qubit = 0
  for pauli in paulis:
    if pauli == "X":
      cirq.h(qubit)
    elif pauli == "Y":
      cirq.sdg(qubit)
      cirq.h(qubit)
    qubit += 1
  cirq.measure_all()
  return cirq

# Calculating the Energy for a parameter adjustment amount.
def GetEnergy(base_cirq, paramValues, backend):
  energies = []
  for pauliString in measurements:
    measured_cirq = ApplyPauliMeasurements(base_cirq, paramValues, pauliString)

    # Transpile and run
    transpiled_circuit = transpile(measured_cirq, backend)
    result = backend.run(transpiled_circuit).result()
    energy = 0
    for key, count in result.get_counts().items():
      energy += ((-1)**key.count("1"))*count

    energies.append(energy/shots)
  return sum([a*b for a,b in zip(energies, coeffs)])


# For trials: run circuit, calc energy, adjust params
def Optimize():
  backend = GenericBackendV2(num_qubits=qubits)
  params = ParameterVector('p', 2 * qubits * layers)
  base_cirq = BuildBaseCircuit(params)

  # Start with a random set of parameters
  paramValues = [random.random()*2*np.pi for _ in range(len(params))]
  base_cirq.assign_parameters(paramValues)

  for trial in range(trials):
    gradients = np.zeros(2*qubits*layers)
    for i in range(len(params)):
      # do + epsilon first
      paramValues[i] += epsilon
      lE = GetEnergy(base_cirq, paramValues, backend)
      # do - epsilon instead
      paramValues[i] -= 2*epsilon
      rE = GetEnergy(base_cirq, paramValues, backend)
      # calculate gradient and reset the current parameter
      gradients[i] = (lE- rE)/(2*epsilon)
      paramValues[i] += epsilon
    paramValues -= learning_rate*gradients
    print("Energy: ", GetEnergy(base_cirq, paramValues, backend))

  # Show the final set of parameters
  print("Final parameter list:", paramValues)
  print("Resulting in energy: ", GetEnergy(base_cirq,paramValues, backend))


Optimize()





┌──────────┐┌──────────┐     ┌──────────┐┌──────────┐     
q_0: ┤ Rx(p[0]) ├┤ Ry(p[1]) ├──■──┤ Rx(p[4]) ├┤ Ry(p[5]) ├──■──
     ├──────────┤├──────────┤┌─┴─┐├──────────┤├──────────┤┌─┴─┐
q_1: ┤ Rx(p[2]) ├┤ Ry(p[3]) ├┤ X ├┤ Rx(p[6]) ├┤ Ry(p[7]) ├┤ X ├
     └──────────┘└──────────┘└───┘└──────────┘└──────────┘└───┘

/usr/local/lib/python3.10/dist-packages/qiskit/providers/fake_provider/generic_backend_v2.py:506: RuntimeWarning: Aer not found using BasicSimulator and no noise
  warnings.warn("Aer not found using BasicSimulator and no noise", RuntimeWarning)


Energy:  0.7548828125
Energy:  0.759765625
Energy:  0.634765625
Energy:  0.6162109375
Energy:  0.623046875
Energy:  0.5263671875
Energy:  0.4189453125
Energy:  0.41015625
Energy:  0.267578125
Energy:  0.1494140625
Energy:  -0.0078125
Energy:  -0.2529296875
Energy:  -0.3740234375
Energy:  -0.6328125
Energy:  -0.91015625
Energy:  -1.0673828125
Energy:  -1.2421875
Energy:  -1.44140625
Energy:  -1.5390625
Energy:  -1.6396484375
Energy:  -1.7734375
Energy:  -1.8193359375
Energy:  -1.8037109375
Energy:  -1.892578125
Energy:  -1.919921875
Energy:  -1.9482421875
Energy:  -1.9404296875
Energy:  -1.955078125
Energy:  -1.962890625
Energy:  -1.9697265625
Energy:  -1.982421875
Energy:  -1.9853515625
Energy:  -1.9853515625
Energy:  -1.9873046875
Energy:  -1.994140625
Energy:  -1.9873046875
Energy:  -1.990234375
Energy:  -1.99609375
Energy:  -1.9970703125
Energy:  -1.9951171875
Energy:  -1.994140625
Energy:  -1.99609375
Energy:  -1.998046875
Energy:  -1.9990234375
Energy:  -2.0
Energy:  -1.9990234375